### Applied Data Science Capstone - Week 3 Assignment, Segmenting and Clustering Neighborhoods in Toronto

*by Chufan Wu*

*Feb.2021*

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.DataFrame({'PostalCode': ['M5G', 'M2H', 'M4B', 'M1J', 'M4G', 'M4M', 'M1R', 'M9V', 'M9L', 'M5V', 'M1B', 'M5A'], 
                  'Borough': ['Downtown Toronto', 'North York', 'East York', 'Scarborough', 'East York', 'East Toronto', 'Scarborough', 'Etobicoke', 'North York', 'Downtown Toronto', 'Scarborough', 'Downtown Toronto'], 
                  'Neighborhood': ['Central Bay Street', 'Hillcrest Village', 'Parkview Hill, Woodbine Gardens', 'Scarborough Village', 'Leaside', 'Studio District', 'Wexford, Maryvale', 'South Steeles, Silverstone, Humbergate, Jamestown, Mount Olive, Beaumond Heights, Thistletown, Albion Gardens', 'Humber Summit', 'CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport', 'Malvern, Rouge', 'Regent Park, Harbourfront']})
df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."


*Data Source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M*

In [4]:
df.shape

(12, 3)